In [1]:
import os
os.chdir("..")
%pwd

'c:\\ML\\NLP-Text-Summarization'

In [2]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class DataValidationEntity:
    root_dir: Path
    status_file: Path
    required_folders: list

In [3]:
from src.constants import *
from src.utils.common import CommonUtils
from box import ConfigBox
from src.custom_exception import CustomException
import sys
class ConfigurationManager:
    def __init__(self, config_path=CONFIG_FILE_PATH, params_path = PARAMS_FILE_PATH):
        self.config: ConfigBox = CommonUtils.read_yaml(config_path)
        self.params: ConfigBox = CommonUtils.read_yaml(params_path)

        CommonUtils.create_directories([self.config.artifacts_root])

    def data_validation_configuration(self) -> DataValidationEntity:
        try:
            config = self.config.data_validation
            CommonUtils.create_directories([config.root_dir])
            return DataValidationEntity(
                root_dir=config.root_dir,
                status_file=config.status_file,
                required_folders=config.required_folders
            )
        except Exception as exp:
            raise CustomException(exp, sys)

In [6]:
class DataValiadtion:
    def __init__(self, config: DataValidationEntity):
        self.config = config

    
    def validate_all_files_exist(self)-> bool:
        try:
            validation_status = None

            all_files = os.listdir(os.path.join("artifacts","data_ingestion","samsum_dataset"))

            for file in all_files:
                if file not in self.config.required_folders:
                    validation_status = False
                    with open(self.config.status_file, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.status_file, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as exp:
            raise CustomException(exp, sys)

In [7]:
try:
    config_manager_obj = ConfigurationManager()
    data_validation_config = config_manager_obj.data_validation_configuration()
    data_validation_obj = DataValiadtion(data_validation_config)
    data_validation_obj.validate_all_files_exist()
except Exception as exp:
    raise CustomException(exp, sys)